# Cassie FloodModule Version 0
- Flood Frequency 
- Flood Pronearea
- Slope Flat area from SRTM
- EAS Landcover as display layer
- Google Open Building as display layer

In [ ]:
# import ee
# import geemap
# import datetime
# import ipywidgets as widgets
# from IPython.display import display
# import re  # Import regex module

# # Initialize Earth Engine API
# ee.Initialize()

# # Create map
# Map = geemap.Map()

# # Load GAUL dataset and filter for Guyana
# gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
# guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))

# # Get list of available cities in Guyana
# raw_city_list = guyana.aggregate_array('ADM1_NAME').getInfo()

# # Clean region names by removing "(region N°X)"
# city_list = [re.sub(r"\s*\(region N°\d+\)", "", name) for name in raw_city_list]

# # Define dynamic time period
# start_date = datetime.date(1984, 3, 16)
# end_date = datetime.date(2024, 12, 31)

# # Convert to ee.Date objects
# start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
# end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# # Load datasets
# jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
# srtm = ee.Image("USGS/SRTMGL1_003")
# worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().select("Map")  # Fixed selection issue
# buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons")  # Open Buildings dataset

# # Interactive widgets
# city_selector = widgets.Dropdown(
#     options=city_list,
#     value=city_list[0],
#     description="City:",
# )
# start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
# end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
# low_lying_input = widgets.BoundedFloatText(description="Low-Lying Level (m)", value=10, min=0, max=100, step=1)
# base_map_selector = widgets.Dropdown(
#     options=["SATELLITE", "OSM"],
#     value="SATELLITE",
#     description="Base Map:",
# )
# run_button = widgets.Button(description="Run Analysis", button_style="success")


# def update_map(change):
#     selected_city = city_selector.value
    
#     # Match the cleaned city name with the original dataset
#     raw_city_name = next(name for name in raw_city_list if selected_city in name)
#     roi = guyana.filter(ee.Filter.eq('ADM1_NAME', raw_city_name)).geometry()
    
#     start_date = start_date_picker.value
#     end_date = end_date_picker.value
#     low_lying_threshold = low_lying_input.value  # Get updated low-lying level
#     selected_basemap = base_map_selector.value  # Get selected base map
    
#     # Convert to ee.Date objects
#     start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
#     end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)
    
#     # Filter datasets to ROI
#     jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
#     permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
#     jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
    
#     # Calculate flood frequency
#     def calculate_flood_frequency(collection):
#         def add_bands(img):
#             obs = img.gt(0).rename("obs")
#             water = img.select("water").eq(2).rename("water")
#             return img.addBands([obs, water])
        
#         collection = collection.map(add_bands)
#         total_obs = collection.select("obs").sum().rename("total_obs")
#         total_water = collection.select("water").sum().rename("total_water")
#         flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
#         return flood_frequency.updateMask(flood_frequency.neq(0))
    
#     flood_frequency = calculate_flood_frequency(jrc_filtered)
#     slope = ee.Terrain.slope(srtm).rename("slope")
#     flat_area = slope.lt(5).rename("flat_area")
#     low_lying = srtm.lt(low_lying_threshold).rename("low_lying")
#     flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
    
#     # Convert building polygons to centroids
#     building_centroids = buildings.filterBounds(roi).map(lambda f: f.centroid())

#     # Count flooded buildings
#     flooded_buildings = building_centroids.filterBounds(flood_prone_area.geometry())
#     flooded_building_count = flooded_buildings.size().getInfo()

#     # Landcover visualization
#     landcover_roi = worldcover.clip(roi)
#     landcover_vis = {
#         "bands": ["Map"],
#         "min": 10,
#         "max": 100,
#         "palette": [
#             "#006400", "#ffbb22", "#ffff4c", "#f096ff", "#fa0000",
#             "#b4b4b4", "#f0f0f0", "#0064c8", "#0096a0", "#00cf75", "#fae6a0"
#         ]
#     }
    
#     # Remove previous layers safely
#     layer_names = [
#         "Flood Frequency",
#         "Flood-Prone Areas",
#         "SRTM DEM",
#         "Slope",
#         "Land Use (ESA WorldCover)",
#         "Buildings",
#         "Flooded Buildings"
#     ]
#     for layer_name in layer_names:
#         try:
#             Map.remove_layer(layer_name)
#         except Exception:
#             pass  # Ignore errors if the layer is not found
    
#     # Set the selected base map
#     Map.add_basemap(selected_basemap)
#     Map.centerObject(roi, 8)
    
#     # Visualization parameters
#     dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
#     slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
#     flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
#     flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}
#     building_vis = {"color": "blue"}
#     flooded_building_vis = {"color": "red"}

#     # Add layers
#     Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
#     Map.addLayer(slope.clip(roi), slope_vis, "Slope")
#     Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, "Flood Frequency")
#     Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, "Flood-Prone Areas")
#     Map.addLayer(landcover_roi, landcover_vis, "Land Use (ESA WorldCover)")
#     Map.addLayer(building_centroids, building_vis, "Buildings")
#     Map.addLayer(flooded_buildings, flooded_building_vis, "Flooded Buildings")

#     # Add flooded building count as text overlay (legend)
#     Map.add_text(
#         text=f"Flooded Buildings: {flooded_building_count}",
#         position="topright",
#         font_size=14,
#         text_color="white",
#         background_color="black",
#         name="Flooded Buildings Count"
#     )


#     add_legend()

# def add_legend():
#     legend_dict = {
#         "Trees": "#006400",
#         "Shrubland": "#ffbb22",
#         "Grassland": "#ffff4c",
#         "Cropland": "#f096ff",
#         "Built-up": "#fa0000",
#         "Bare / Sparse Veg": "#b4b4b4",
#         "Snow & Ice": "#f0f0f0",
#         "Permanent Water": "#0064c8",
#         "Herbaceous Wetland": "#0096a0",
#         "Mangroves": "#00cf75",
#         "Moss / Lichen": "#fae6a0",
#     }

#     Map.add_legend(title="ESA WorldCover 2020", legend_dict=legend_dict)


# # Set the button action
# run_button.on_click(update_map)

# # Display the widgets
# display(city_selector, start_date_picker, end_date_picker, low_lying_input, base_map_selector, run_button)

# # Ensure the base map is added initially
# Map.addLayerControl()
# Map


Dropdown(description='City:', options=('Barima Waini', 'Cuyuni/mazaruni', 'Demerara Mahaica', 'East Berbice/co…

DatePicker(value=datetime.date(1984, 3, 16), description='Start Date', step=1)

DatePicker(value=datetime.date(2024, 12, 31), description='End Date', step=1)

BoundedFloatText(value=10.0, description='Low-Lying Level (m)', step=1.0)

Dropdown(description='Base Map:', options=('SATELLITE', 'OSM'), value='SATELLITE')

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Cassie FloodModule Version 1
- Flood Frequency 
- Flood Pronearea
- Slope Flat area from SRTM
- EAS Landcover as display layer
- Google Open Building as display layer
- display building as Feature 

In [1]:
import ee
import geemap
import datetime
import ipywidgets as widgets
from IPython.display import display
import re  # Import regex module

# Initialize Earth Engine API
ee.Initialize()

# Create map
Map = geemap.Map()

# Load GAUL dataset and filter for Guyana
gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))

# Get list of available cities in Guyana
raw_city_list = guyana.aggregate_array('ADM1_NAME').getInfo()

# Clean region names by removing "(region N°X)"
city_list = [re.sub(r"\s*\(region N°\d+\)", "", name) for name in raw_city_list]

# Define dynamic time period
start_date = datetime.date(1984, 3, 16)
end_date = datetime.date(2024, 12, 31)

# Convert to ee.Date objects
start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# Load datasets
jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
srtm = ee.Image("USGS/SRTMGL1_003")
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().select("Map")  # Fixed selection issue
buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons")  # Open Buildings dataset

# Interactive widgets
city_selector = widgets.Dropdown(
    options=city_list,
    value=city_list[0],
    description="City:",
)
start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
low_lying_input = widgets.BoundedFloatText(description="Low-Lying Level (m)", value=10, min=0, max=100, step=1)
base_map_selector = widgets.Dropdown(
    options=["SATELLITE", "OSM"],
    value="SATELLITE",
    description="Base Map:",
)
run_button = widgets.Button(description="Run Analysis", button_style="success")


def update_map(change):
    selected_city = city_selector.value

    # Match the cleaned city name with the original dataset
    raw_city_name = next(name for name in raw_city_list if selected_city in name)
    roi = guyana.filter(ee.Filter.eq('ADM1_NAME', raw_city_name)).geometry()

    start_date = start_date_picker.value
    end_date = end_date_picker.value
    low_lying_threshold = low_lying_input.value  # Get updated low-lying level
    selected_basemap = base_map_selector.value  # Get selected base map

    # Convert to ee.Date objects
    start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
    end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

    # Filter datasets to ROI
    jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
    permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
    jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))

    # Calculate flood frequency
    def calculate_flood_frequency(collection):
        def add_bands(img):
            obs = img.gt(0).rename("obs")
            water = img.select("water").eq(2).rename("water")
            return img.addBands([obs, water])

        collection = collection.map(add_bands)
        total_obs = collection.select("obs").sum().rename("total_obs")
        total_water = collection.select("water").sum().rename("total_water")
        flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
        return flood_frequency.updateMask(flood_frequency.neq(0))

    flood_frequency = calculate_flood_frequency(jrc_filtered)
    slope = ee.Terrain.slope(srtm).rename("slope")
    flat_area = slope.lt(5).rename("flat_area")
    low_lying = srtm.lt(low_lying_threshold).rename("low_lying")
    flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))

    ### ---------------------- Building Analysis ---------------------- ###
    # **Use original building polygons instead of centroids**
    building_polygons = buildings.filterBounds(roi)
    # Count flooded buildings (filter original polygons)
    flooded_buildings = building_polygons.filterBounds(flood_prone_area.geometry())

    # Count flooded buildings
    building_centroids = buildings.filterBounds(roi).map(lambda f: f.centroid())
    flooded_bld = building_centroids.filterBounds(flood_prone_area.geometry())
    flooded_building_count = flooded_bld.size().getInfo()

    # Landcover visualization
    landcover_roi = worldcover.clip(roi)
    landcover_vis = {
        "bands": ["Map"],
        "min": 10,
        "max": 100,
        "palette": [
            "#006400", "#ffbb22", "#ffff4c", "#f096ff", "#fa0000",
            "#b4b4b4", "#f0f0f0", "#0064c8", "#0096a0", "#00cf75", "#fae6a0"
        ]
    }

    # Remove previous layers safely
    layer_names = [
        "Flood Frequency",
        "Flood-Prone Areas",
        "SRTM DEM",
        "Slope",
        "Land Use (ESA WorldCover)",
        "Buildings",
        "Flooded Buildings"
    ]
    for layer_name in layer_names:
        try:
            Map.remove_layer(layer_name)
        except Exception:
            pass  # Ignore errors if the layer is not found

    # Set the selected base map
    Map.add_basemap(selected_basemap)
    Map.centerObject(roi, 8)

    # Visualization parameters
    dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
    slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
    flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
    flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}
    building_vis = {"color": "blue", "fillColor": "blue", "width": 1}
    flooded_building_vis = {"color": "red", "fillColor": "red", "width": 1}
    # count_label_vis = {"color": "black", "pointSize": 8}

    
    # Add layers
    Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
    Map.addLayer(slope.clip(roi), slope_vis, "Slope")
    Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, "Flood Frequency")
    Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, "Flood-Prone Areas")
    Map.addLayer(landcover_roi, landcover_vis, "Land Use (ESA WorldCover)")
    Map.addLayer(building_polygons, building_vis, "Buildings")
    Map.addLayer(flooded_buildings, flooded_building_vis, "Flooded Buildings")
    
    # Add flooded building count as text overlay (legend)
    Map.add_text(
        text=f"Flooded Buildings: {flooded_building_count}",
        position="topright",
        font_size=14,
        text_color="white",
        background_color="black",
        name="Flooded Buildings Count"
    )

    add_legend()


def add_legend():
    legend_dict = {
        "Trees": "#006400",
        "Shrubland": "#ffbb22",
        "Grassland": "#ffff4c",
        "Cropland": "#f096ff",
        "Built-up": "#fa0000",
        "Bare / Sparse Veg": "#b4b4b4",
        "Snow & Ice": "#f0f0f0",
        "Permanent Water": "#0064c8",
        "Herbaceous Wetland": "#0096a0",
        "Mangroves": "#00cf75",
        "Moss / Lichen": "#fae6a0",
    }
    Map.add_legend(title="ESA WorldCover 2020", legend_dict=legend_dict)


# Set the button action
run_button.on_click(update_map)

# Display the widgets
display(city_selector, start_date_picker, end_date_picker, low_lying_input, base_map_selector, run_button)

# Ensure the base map is added initially
Map.addLayerControl()
Map


Dropdown(description='City:', options=('Barima Waini', 'Cuyuni/mazaruni', 'Demerara Mahaica', 'East Berbice/co…

DatePicker(value=datetime.date(1984, 3, 16), description='Start Date', step=1)

DatePicker(value=datetime.date(2024, 12, 31), description='End Date', step=1)

BoundedFloatText(value=10.0, description='Low-Lying Level (m)', step=1.0)

Dropdown(description='Base Map:', options=('SATELLITE', 'OSM'), value='SATELLITE')

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Flooded Area

In [ ]:
# import ee
# import geemap
# import datetime
# import ipywidgets as widgets
# from IPython.display import display
# import re  # Import regex module

# # Initialize Earth Engine API
# ee.Initialize()

# # Create map
# Map = geemap.Map()

# # Load GAUL dataset and filter for Guyana
# gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
# guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))

# # Get list of available cities in Guyana
# raw_city_list = guyana.aggregate_array('ADM1_NAME').getInfo()

# # Clean region names by removing "(region N°X)"
# city_list = [re.sub(r"\s*\(region N°\d+\)", "", name) for name in raw_city_list]

# # Define dynamic time period
# start_date = datetime.date(1984, 3, 16)
# end_date = datetime.date(2024, 12, 31)

# # Convert to ee.Date objects
# start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
# end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# # Load datasets
# jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
# srtm = ee.Image("USGS/SRTMGL1_003")
# worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().select("Map")  # Fixed selection issue
# buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons")  # Open Buildings dataset

# # Interactive widgets
# city_selector = widgets.Dropdown(
#     options=city_list,
#     value=city_list[0],
#     description="City:",
# )
# start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
# end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
# low_lying_input = widgets.BoundedFloatText(description="Low-Lying Level (m)", value=10, min=0, max=100, step=1)
# base_map_selector = widgets.Dropdown(
#     options=["SATELLITE", "OSM"],
#     value="SATELLITE",
#     description="Base Map:",
# )
# run_button = widgets.Button(description="Run Analysis", button_style="success")


# def update_map(change):
#     selected_city = city_selector.value

#     # Match the cleaned city name with the original dataset
#     raw_city_name = next(name for name in raw_city_list if selected_city in name)
#     roi = guyana.filter(ee.Filter.eq('ADM1_NAME', raw_city_name)).geometry()

#     start_date = start_date_picker.value
#     end_date = end_date_picker.value
#     low_lying_threshold = low_lying_input.value  # Get updated low-lying level
#     selected_basemap = base_map_selector.value  # Get selected base map

#     # Convert to ee.Date objects
#     start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
#     end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

#     # Filter datasets to ROI
#     jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
#     permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
#     jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))

#     # Calculate flood frequency
#     def calculate_flood_frequency(collection):
#         def add_bands(img):
#             obs = img.gt(0).rename("obs")
#             water = img.select("water").eq(2).rename("water")
#             return img.addBands([obs, water])

#         collection = collection.map(add_bands)
#         total_obs = collection.select("obs").sum().rename("total_obs")
#         total_water = collection.select("water").sum().rename("total_water")
#         flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
#         return flood_frequency.updateMask(flood_frequency.neq(0))

#     flood_frequency = calculate_flood_frequency(jrc_filtered)
#     slope = ee.Terrain.slope(srtm).rename("slope")
#     flat_area = slope.lt(5).rename("flat_area")
#     low_lying = srtm.lt(low_lying_threshold).rename("low_lying")
#     flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))

#     worldcover_clipped = worldcover.clip(flood_prone_area.geometry())

#     landcover_histogram = worldcover_clipped.reduceRegion(
#         reducer=ee.Reducer.frequencyHistogram(),
#         geometry=flood_prone_area.geometry().bounds(),
#         scale=30,
#         maxPixels=1e13
#     ).get("Map")


#     ### ---------------------- Building Analysis ---------------------- ###
#     # **Use original building polygons instead of centroids**
#     building_polygons = buildings.filterBounds(roi)
#     # Count flooded buildings (filter original polygons)
#     flooded_buildings = building_polygons.filterBounds(flood_prone_area.geometry())

#     # Count flooded buildings
#     building_centroids = buildings.filterBounds(roi).map(lambda f: f.centroid())
#     flooded_bld = building_centroids.filterBounds(flood_prone_area.geometry())
#     flooded_building_count = flooded_bld.size().getInfo()

#     # Landcover visualization
#     landcover_roi = worldcover.clip(roi)
#     landcover_vis = {
#         "bands": ["Map"],
#         "min": 10,
#         "max": 100,
#         "palette": [
#             "#006400", "#ffbb22", "#ffff4c", "#f096ff", "#fa0000",
#             "#b4b4b4", "#f0f0f0", "#0064c8", "#0096a0", "#00cf75", "#fae6a0"
#         ]
#     }

#     # Remove previous layers safely
#     layer_names = [
#         "Flood Frequency",
#         "Flood-Prone Areas",
#         "SRTM DEM",
#         "Slope",
#         "Land Use (ESA WorldCover)",
#         "Buildings",
#         "Flooded Buildings"
#     ]
#     for layer_name in layer_names:
#         try:
#             Map.remove_layer(layer_name)
#         except Exception:
#             pass  # Ignore errors if the layer is not found

#     # Set the selected base map
#     Map.add_basemap(selected_basemap)
#     Map.centerObject(roi, 8)

#     # Visualization parameters
#     dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
#     slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
#     flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
#     flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}
#     building_vis = {"color": "blue", "fillColor": "blue", "width": 1}
#     flooded_building_vis = {"color": "red", "fillColor": "red", "width": 1}
#     # count_label_vis = {"color": "black", "pointSize": 8}

    
#     # Add layers
#     Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
#     Map.addLayer(slope.clip(roi), slope_vis, "Slope")
#     Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, "Flood Frequency")
#     Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, "Flood-Prone Areas")
#     Map.addLayer(landcover_roi, landcover_vis, "Land Use (ESA WorldCover)")
#     Map.addLayer(building_polygons, building_vis, "Buildings")
#     Map.addLayer(flooded_buildings, flooded_building_vis, "Flooded Buildings")
    
#     # Add flooded building count as text overlay (legend)
#     Map.add_text(
#         text=f"Flooded Buildings: {flooded_building_count}",
#         position="topright",
#         font_size=8,
#         text_color="white",
#         background_color="black",
#         name="Flooded Buildings Count"
#     )

#     if landcover_histogram:
#         legend_dict = landcover_histogram.getInfo()
#         flooded_area_table = "Flooded Area per LULC (sq km):\n"
#         for lc_class, area in legend_dict.items():
#             flooded_area_table += f"{lc_class}: {area / 1e6:.2f} km²\n"

#     # Display text on the map
#     Map.add_text(
#         text=flooded_area_table,
#         position="bottomright",
#         font_size=8,
#         text_color="white",
#         background_color="black",
#         name="Flooded Area Table"
#     )

#     add_legend()


# def add_legend():
#     legend_dict = {
#         "Trees": "#006400",
#         "Shrubland": "#ffbb22",
#         "Grassland": "#ffff4c",
#         "Cropland": "#f096ff",
#         "Built-up": "#fa0000",
#         "Bare / Sparse Veg": "#b4b4b4",
#         "Snow & Ice": "#f0f0f0",
#         "Permanent Water": "#0064c8",
#         "Herbaceous Wetland": "#0096a0",
#         "Mangroves": "#00cf75",
#         "Moss / Lichen": "#fae6a0",
#     }
#     Map.add_legend(title="ESA WorldCover 2020", legend_dict=legend_dict)


# # Set the button action
# run_button.on_click(update_map)

# # Display the widgets
# display(city_selector, start_date_picker, end_date_picker, low_lying_input, base_map_selector, run_button)

# # Ensure the base map is added initially
# Map.addLayerControl()
# Map


Dropdown(description='City:', options=('Barima Waini', 'Cuyuni/mazaruni', 'Demerara Mahaica', 'East Berbice/co…

DatePicker(value=datetime.date(1984, 3, 16), description='Start Date', step=1)

DatePicker(value=datetime.date(2024, 12, 31), description='End Date', step=1)

BoundedFloatText(value=10.0, description='Low-Lying Level (m)', step=1.0)

Dropdown(description='Base Map:', options=('SATELLITE', 'OSM'), value='SATELLITE')

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

EEException: Image.clip: The geometry for image clipping must be bounded.

# Cleaner Version exclude UI

In [ ]:
import ee
import datetime
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

# Initialize Earth Engine API
ee.Initialize(project = 'servir-ee')

print("✅ Earth Engine initialized.")

# ----------------------------- #
#       USER DEFINED AOI        #
# ----------------------------- #

# OPTION 1: Define a BBox as AOI (min_lon, min_lat, max_lon, max_lat)
use_bbox = True  # Set to False to use a KML file
bbox = [-58.024429,6.729363,-57.935249,6.763412]  # Example bounding box

# OPTION 2: Load a KML file as AOI
use_kml = not use_bbox  # Switch to True if using KML
# use_kml = True
kml_file = "D:/SIG/G_CAS/01_Bathymetry/00_SOURCE1/EPSG4326/_use/aoi.kml"  # Replace with actual KML file path

# Function to extract geometry from KML
import geopandas as gpd
from shapely.geometry import shape

def load_kml_geometry(kml_file):
    print(f"📂 Loading KML file: {kml_file}")

    try:
        # Read KML using GeoPandas
        gdf = gpd.read_file(kml_file, driver="KML")
        print(f"✅ KML file loaded successfully. Found {len(gdf)} features.")

        # Extract first feature geometry as GeoJSON
        geom = gdf.geometry.iloc[0]  # Get first feature
        geojson_geom = shape(geom).__geo_interface__  # Convert to GeoJSON format

        print("✅ Geometry extracted from KML.")
        return ee.Geometry(geojson_geom)  # Convert to Earth Engine Geometry

    except Exception as e:
        print(f"❌ Error loading KML: {e}")
        return None

# ----------------------------- #
#   TIME RANGE & PARAMETERS     #
# ----------------------------- #

# Time range for analysis
start_date = datetime.date(1984, 3, 16)
end_date = datetime.date(2024, 12, 31)
low_lying_threshold = 10  # Low-lying threshold (m)

# Convert to Earth Engine date objects
start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

print("✅ Time range set:", start_date, "to", end_date)

# ----------------------------- #
#       DEFINE REGION (AOI)     #
# ----------------------------- #

roi = None  # Initialize ROI as None

if use_kml:  # If using KML, load AOI from KML file
    roi = load_kml_geometry(kml_file)
elif use_bbox:  # If using BBOX, define AOI as a bounding box
    roi = ee.Geometry.BBox(*bbox)  # Convert list to bbox geometry

# 🚨 Ensure ROI is valid before proceeding
if roi is None:
    raise ValueError("❌ ERROR: Region of Interest (ROI) is not defined! Check AOI selection.")

print("✅ AOI Loaded Successfully:", roi.getInfo())  # Debugging check

# ----------------------------- #
#         LOAD DATASETS         #
# ----------------------------- #

print("⏳ Loading datasets...")

# Satellite Data
jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory").filterBounds(roi).filterDate(start_date_ee, end_date_ee)
srtm = ee.Image("USGS/SRTMGL1_003")
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().select("Map")
buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons")

print("✅ Datasets loaded successfully.")

# Filter permanent water mask
permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
jrc = jrc.map(lambda img: img.updateMask(permanent_water.Not()))

print("✅ Applied water mask.")

# ----------------------------- #
#    FLOOD FREQUENCY ANALYSIS   #
# ----------------------------- #

print("⏳ Calculating flood frequency...")

def calculate_flood_frequency(collection):
    def add_bands(img):
        obs = img.gt(0).rename("obs")
        water = img.select("water").eq(2).rename("water")
        return img.addBands([obs, water])

    collection = collection.map(add_bands)
    total_obs = collection.select("obs").sum().rename("total_obs")
    total_water = collection.select("water").sum().rename("total_water")
    flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
    return flood_frequency.updateMask(flood_frequency.neq(0))

flood_frequency = calculate_flood_frequency(jrc)

print("✅ Flood frequency calculated.")

# ----------------------------- #
#   FLOOD-PRONE AREA DETECTION  #
# ----------------------------- #

print("⏳ Identifying flood-prone areas...")

slope = ee.Terrain.slope(srtm).rename("slope")
flat_area = slope.lt(5).rename("flat_area")
low_lying = srtm.lt(low_lying_threshold).rename("low_lying")
flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))


print("✅ Flood-prone areas identified.")

# Ensure flood-prone area is integer
flood_prone_int = flood_prone_area.multiply(100).toInt()

# Convert to vectors with a tolerance to reduce feature count
flood_prone_fc = ee.FeatureCollection(
    flood_prone_int.reduceToVectors(
        geometryType='polygon',
        reducer=ee.Reducer.countEvery(),
        geometry=roi,  # Ensure processing is within the AOI
        scale=30,
        maxPixels=1e13,
        bestEffort=True,  # Reduce output complexity
        tileScale=2  # Reduce memory usage
    )
).map(lambda f: f.simplify(30))  # Simplify geometry

# Buffer before union (simulate error margin)
flood_prone_fc = flood_prone_fc.map(lambda f: f.buffer(30)).union()

# # Get GeoJSON
# flood_prone_geojson = flood_prone_fc.getInfo()

# # Define output GeoJSON file path
# geojson_filename = "/Users/weraphongsuaruang/Python/Cassie_FloodModule/_result/flood_prone_area1.geojson"

# # Save the GeoJSON data
# with open(geojson_filename, "w") as f:
#     json.dump(flood_prone_geojson, f)

# print(f"✅ Flood-prone area exported as GeoJSON: {geojson_filename}.")



# ----------------------------- #
#   FLOODED BUILDING ANALYSIS   #
# ----------------------------- #

# Ensure flood-prone area is clipped to the AOI before vectorization
flood_prone_clipped = flood_prone_area.clip(roi).toInt()  # Convert to integer

# Convert flood image to vector (only flooded areas within AOI)
flood_prone_vector = flood_prone_clipped.reduceToVectors(
    reducer=ee.Reducer.countEvery(),
    geometry=roi,  # Specify AOI to limit vectorization
    geometryType='polygon',
    scale=30,  # Adjust resolution if needed
    maxPixels=1e8
)

# Ensure the flood-prone area is valid geometry
flood_prone_geom = flood_prone_vector.geometry()

# Filter buildings within AOI
buildings_in_aoi = buildings.filterBounds(roi)

# Count total buildings in AOI
total_buildings = buildings_in_aoi.size().getInfo()
print(f"🔍 Total buildings in AOI: {total_buildings}")

# Filter flooded buildings using the converted geometry
flooded_buildings = buildings_in_aoi.filterBounds(flood_prone_geom)

# Count flooded buildings
flooded_building_count = flooded_buildings.size().getInfo()
print(f"✅ Flooded buildings identified: {flooded_building_count}")


# ----------------------------- #
#  FLOODED AREA PER LULC CLASS  #
# ----------------------------- #

print("⏳ Calculating flooded area per LULC class...")

# Define LULC classes and their names
lulc_mapping = {
    10: "Tree cover",
    20: "Shrubland",
    30: "Grassland",
    40: "Cropland",
    50: "Built-up",
    60: "Bare / sparse vegetation",
    70: "Snow and ice",
    80: "Permanent water bodies",
    90: "Herbaceous wetland",
    95: "Mangroves",
}

lulc_values = list(lulc_mapping.keys())  # ESA WorldCover classes
landcover_masked = worldcover.clip(roi).updateMask(flood_prone_area)
area_km2 = []

for lulc_class in lulc_values:
    class_mask = landcover_masked.eq(lulc_class)
    area_m2 = class_mask.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=30,
        maxPixels=1e13
    ).getInfo()

    flooded_area_km2 = area_m2.get("Map", 0) / 1e6 if area_m2 else 0  # Convert to km²

    # Store in list with LULC class name
    area_km2.append({
        "LULC_Class": lulc_class,
        "LULC_Name": lulc_mapping.get(lulc_class, "Unknown"),
        "Flooded_Area_km²": flooded_area_km2
    })

print("✅ Flooded area per LULC class calculated.")


# # ----------------------------- #
# #         EXPORT RESULTS        #
# # ----------------------------- #

## export to drive 

print('Start Export to drive')
print("⏳ Exporting Flood-Prone area...")
export_task = ee.batch.Export.table.toDrive(
    collection=flood_prone_fc,
    description="FloodProne_Area",
    folder="FloodAnalysis",
    fileFormat="GeoJSON"
)
export_task.start()

print("⏳ Exporting AOI Building...")
export_aoi_bld = ee.batch.Export.table.toDrive(
    collection=buildings_in_aoi,
    description="AOI_Buildings",
    folder="FloodAnalysis",
    fileFormat="GeoJSON"
)

export_aoi_bld.start()

print("⏳ Exporting AOI Building...")
export_fld_bld = ee.batch.Export.table.toDrive(
    collection=flooded_buildings,
    description="Flooded_Buildings",
    folder="FloodAnalysis",
    fileFormat="GeoJSON"
)

export_fld_bld.start()

print("⏳ Exporting Flooded area per LULC...")

csv_filename = "/Users/weraphongsuaruang/Python/Cassie_FloodModule/_result/flooded_area_per_lulc.csv"
csv_data = pd.DataFrame(area_km2)
csv_data.to_csv(csv_filename, index=False)
print(f"✅ Flooded area per LULC saved as CSV: {csv_filename}.")


# --- Export to Google Drive ---
# Convert DataFrame to a FeatureCollection for GEE export
features = []
for row in area_km2:
    feature = ee.Feature(None, row)  # Convert each row to an EE Feature
    features.append(feature)

# Create FeatureCollection
fc = ee.FeatureCollection(features)

# Define Google Drive Export Task
task = ee.batch.Export.table.toDrive(
    collection=fc,
    description="Flooded_Area_Per_LULC",
    fileFormat="CSV",
    folder="FloodAnalysis" 
)

# Start export task
task.start()



# ----------------------------- #
#         PRINT RESULTS         #
# ----------------------------- #

print("\n=== FLOOD ANALYSIS RESULTS ===")
print(f"Flooded Buildings: {flooded_building_count}")
# print("Flooded Area (km²) by LULC Class:")
# # for lulc_class, area in area_km2.items():
# #     print(f"  Class {lulc_class}: {area:.2f} km²")
for entry in area_km2:
    print(f"  Class {entry['LULC_Class']}: {entry['Flooded_Area_km²']:.2f} km²")


# print("\n🚀 Exporting results...")
print(f"- Flooded buildings saved as Shapefile & GeoJSON in Google Drive (folder: 'FloodAnalysis').")
# print(f"- Flooded area per LULC saved as CSV: {csv_filename}.")



In [ ]:
### export geojson to Local machine ------------ NOT WORK size limit issue  -----------------
# # Define file paths
# aoi_bld_filename = "/Users/weraphongsuaruang/Python/Cassie_FloodModule/_result/aoi_bld.geojson"
# flooded_bld_filename = "/Users/weraphongsuaruang/Python/Cassie_FloodModule/_result/flooded_bld.geojson"

# # Convert FeatureCollections to client-side GeoJSON dictionaries
# aoi_bld_geojson = buildings_in_aoi.getInfo()  # Convert to Python dict
# flooded_bld_geojson = flooded_buildings.getInfo()  # Convert to Python dict

# # Save AOI buildings as GeoJSON
# with open(aoi_bld_filename, "w") as f:
#     json.dump(aoi_bld_geojson, f)

# print(f"✅ AOI buildings exported as GeoJSON: {aoi_bld_filename}")

# # Save Flooded buildings as GeoJSON
# with open(flooded_bld_filename, "w") as f:
#     json.dump(flooded_bld_geojson, f)

# print(f"✅ Flooded buildings exported as GeoJSON: {flooded_bld_filename}")

# ##---------------------------------------------------------------------------------------------------##